In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv(
    "/Users/thrilok/Library/CloudStorage/OneDrive-SharedLibraries-Mantracollabtechnologyandservices/Mantra Collab Broko - General/Broko-Thrilok/knn/output.csv"
)

In [3]:
data["Community"].unique()

array(['city centre', 'hurontario', 'central erin mills', 'erindale',
       'port credit', 'creditview', 'mississauga valleys', 'cooksville',
       'rathwood', 'erin mills', 'applewood', 'meadowvale', 'malton',
       'clarkson', 'sheridan', 'lakeview', 'streetsville',
       'churchill meadows', 'east credit', 'meadowvale village',
       'mineola', 'fairview', 'northeast', 'dixie', 'sheridan park'],
      dtype=object)

In [4]:
data = data[data["Community"] == "port credit"]
data = data.dropna()
len(data)

126

In [5]:
data.dtypes

Municipality                      object
Municipality_District             object
Community                         object
Address                           object
ML_Number                         object
lat                              float64
lng                              float64
calculated_Approx_Age             object
Sold_Month                        object
latest_tax                       float64
Washrooms                          int64
Bedrooms                           int64
Bedrooms_                          int64
Adjusted_Total_Parking_Spaces      int64
Parking_Category                  object
Modified_Balcony                  object
Approx_Square_Footage             object
FLOOR_NUMBER                      object
Current_Price                    float64
dtype: object

In [6]:
data.columns

Index(['Municipality', 'Municipality_District', 'Community', 'Address',
       'ML_Number', 'lat', 'lng', 'calculated_Approx_Age', 'Sold_Month',
       'latest_tax', 'Washrooms', 'Bedrooms', 'Bedrooms_',
       'Adjusted_Total_Parking_Spaces', 'Parking_Category', 'Modified_Balcony',
       'Approx_Square_Footage', 'FLOOR_NUMBER', 'Current_Price'],
      dtype='object')

In [7]:
columns = [
    "lat",
    "lng",
    "calculated_Approx_Age",
    "latest_tax",
    "Washrooms",
    "Bedrooms",
    "Bedrooms_",
    "Adjusted_Total_Parking_Spaces",
    "Parking_Category",
    "Modified_Balcony",
    "Approx_Square_Footage",
    "FLOOR_NUMBER",
    "Current_Price",
    "Address",
    "Sold_Month",
]
categorical_columns = [
    "Washrooms",
    "Bedrooms",
    "Adjusted_Total_Parking_Spaces",
    "Parking_Category",
    "Modified_Balcony",
    "FLOOR_NUMBER",
    "calculated_Approx_Age",
    "Approx_Square_Footage",
    # "Address",
    "Bedrooms_",
]
numerical_columns = [
    "latest_tax",
]

In [8]:
df = data[columns]

In [9]:
from datetime import datetime, timedelta

df["Sold_Month"] = pd.to_datetime(df["Sold_Month"], format="%y-%b")

# Filter out rows with NaT (parsing errors)
df = df.dropna(subset=["Sold_Month"])

df["Sold_Month"]

/var/folders/4d/gx0lrt1n0l34spb_k9cnkfpr0000gn/T/ipykernel_5555/1530680431.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sold_Month"] = pd.to_datetime(df["Sold_Month"], format="%y-%b")


10     2023-10-01
78     2023-05-01
150    2023-04-01
151    2023-07-01
290    2023-07-01
          ...    
5309   2023-12-01
5322   2023-12-01
5332   2023-12-01
5357   2023-11-01
5436   2023-12-01
Name: Sold_Month, Length: 126, dtype: datetime64[ns]

In [16]:
timedelta_values = [30, 60, 90, 120, 150, 180]

# Flag to indicate if any data is found
data_found = False
# current_date = datetime.now().date()
current_date = datetime(2023, 12, 1).date()
for delta in timedelta_values:
    # Calculate start_date based on the current delta
    start_date = current_date - timedelta(days=delta)
    # Filter data for sold months within the specified time range
    filtered_df = df[
        (df["Sold_Month"] >= np.datetime64(start_date))
        & (df["Sold_Month"] <= np.datetime64(current_date))
    ]

    if len(filtered_df) != 0:
        # Select relevant columns for modeling
        X = filtered_df.drop(["Address", "Sold_Month"], axis=1)
        X = pd.get_dummies(X, columns=categorical_columns, dtype=int)

        # Fit the nearest neighbors model
        knn = NearestNeighbors(n_neighbors=5)
        knn.fit(X)

        # Set the data_found flag to True
        data_found = True

        # Optionally, you can break out of the loop if you only want to use the first non-empty result
        break
# Define the query house
house = pd.DataFrame(df.iloc[0]).T
query_house = house.drop(["Address", "Sold_Month"], axis=1)
query_house = pd.get_dummies(query_house, columns=categorical_columns, dtype=int)
# Ensure query_house has the same columns as X
missing_columns = set(X.columns) - set(query_house.columns)
print(f"missing columns: {missing_columns}")
print(f"X.columns: {X.columns}")
print(f"query_house.columns: {query_house.columns}")
for column in missing_columns:
    query_house[column] = 0


# pickeld_df = np.zeros((1, len(X.columns)))  # Creating a single-row array of zeros
# query_data = pd.DataFrame(pickeld_df, columns=X.columns)
# query_data.update(query_house)
print(f"The query house: {query_house.shape}")
print(f"The query data: {query_data.shape}")

missing columns: {'FLOOR_NUMBER_5', 'Approx_Square_Footage_1600-1799', 'Approx_Square_Footage_1800-1999', 'calculated_Approx_Age_11-15', 'Approx_Square_Footage_800-899', 'calculated_Approx_Age_16-30', 'Modified_Balcony_encl', 'Washrooms_3', 'Bedrooms_1', 'Approx_Square_Footage_900-999', 'FLOOR_NUMBER_17', 'Approx_Square_Footage_700-799', 'Adjusted_Total_Parking_Spaces_2', 'Approx_Square_Footage_1000-1199', 'Bedrooms_2', 'Approx_Square_Footage_2000-2249', 'FLOOR_NUMBER_8', 'calculated_Approx_Age_0-5', 'Washrooms_1', 'FLOOR_NUMBER_15', 'FLOOR_NUMBER_3'}
X.columns: Index(['lat', 'lng', 'latest_tax', 'Current_Price', 'Washrooms_1',
       'Washrooms_2', 'Washrooms_3', 'Bedrooms_1', 'Bedrooms_2', 'Bedrooms_3',
       'Adjusted_Total_Parking_Spaces_1', 'Adjusted_Total_Parking_Spaces_2',
       'Parking_Category_Underground', 'Modified_Balcony_Open',
       'Modified_Balcony_encl', 'FLOOR_NUMBER_15', 'FLOOR_NUMBER_17',
       'FLOOR_NUMBER_3', 'FLOOR_NUMBER_5', 'FLOOR_NUMBER_6', 'FLOOR_NUMBER

In [11]:
X

,lat,lng,latest_tax,Current_Price,Washrooms_1,Washrooms_2,Washrooms_3,Bedrooms_1,Bedrooms_2,Bedrooms_3,...,calculated_Approx_Age_16-30,Approx_Square_Footage_1000-1199,Approx_Square_Footage_1200-1399,Approx_Square_Footage_1600-1799,Approx_Square_Footage_1800-1999,Approx_Square_Footage_2000-2249,Approx_Square_Footage_700-799,Approx_Square_Footage_800-899,Approx_Square_Footage_900-999,Bedrooms__0
5045,43.555963,-79.582234,4398.880000,7.957312e+05,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
5099,43.553536,-79.587029,4527.478556,9.068333e+05,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
5189,43.553263,-79.582547,7857.620000,2.151977e+06,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1
5224,43.555051,-79.583333,9071.000000,1.401288e+06,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
5288,43.553277,-79.581242,0.000000,1.301196e+06,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
5309,43.553536,-79.587029,7898.590000,1.700000e+06,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
5322,43.553882,-79.582057,3869.950000,5.900000e+05,1,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,1
5332,43.555963,-79.582234,4531.110000,9.750000e+05,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
5357,43.553882,-79.582057,5879.000000,1.246145e+06,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,0,1
5436,43.555963,-79.582234,3136.410000,6.820000e+05,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [12]:
timedelta_values = [30, 60, 90, 120, 180]

# Flag to indicate if any data is found
data_found = False
# current_date = datetime.now().date()
current_date = datetime(2023, 12, 1).date()
for delta in timedelta_values:
    # Calculate start_date based on the current delta
    start_date = current_date - timedelta(days=delta)

    # Filter data for sold months within the specified time range
    filtered_df = df[
        (df["Sold_Month"] >= start_date) & (df["Sold_Month"] <= current_date)
    ]

    print(len(filtered_df))
    # Check if filtered_df is not empty
    if len(filtered_df) != 0:
        # Select relevant columns for modeling
        X = filtered_df.drop(["Address", "Sold_Month"], axis=1)

        # Fit the nearest neighbors model
        knn = NearestNeighbors(n_neighbors=5)
        knn.fit(X)

        # Set the data_found flag to True
        data_found = True

        # Optionally, you can break out of the loop if you only want to use the first non-empty result
        # break
    else:
        pass

# Check if any data is found
if data_found:
    # Transform the query house data
    query_data = pd.DataFrame(query_house, index=[0])

    # Find the nearest neighbors for the query house
    distances, indices = knn.kneighbors(query_data)

    # Display the nearest neighbors
    nearest_neighbors = filtered_df.iloc[indices[0]]
    print(nearest_neighbors)
else:
    print("No samples found within the specified time ranges.")

TypeError: Invalid comparison between dtype=datetime64[ns] and date

In [ ]:
# Filter data for sold months within the last 30 days
current_date = datetime.now().date()
start_date = current_date - timedelta(days=30)
filtered_df = df[(df["Sold_Month"] >= start_date) & (df["Sold_Month"] <= current_date)]

In [ ]:
filtered_df

In [ ]:
# Select relevant columns for modeling
X = filtered_df.drop(["Address", "Sold_Month"], axis=1)

# Fit the nearest neighbors model
knn = NearestNeighbors(
    n_neighbors=5
)  # You can adjust the number of neighbors as needed
knn.fit(X)

# Transform the query house data
query_data = pd.DataFrame(query_house, index=[0])

# Find the nearest neighbors for the query house
distances, indices = knn.kneighbors(query_data)

# Display the nearest neighbors
nearest_neighbors = filtered_df.iloc[indices[0]]
print(nearest_neighbors)

In [ ]:
df = data[columns]

In [ ]:
# Preprocess the data
# Convert categorical features to dummy variables
df = pd.get_dummies(df, columns=categorical_columns, dtype=int)
df

In [ ]:
# Normalize numerical features
df = (df - df.min()) / (df.max() - df.min())
df

In [ ]:
df.describe()

In [ ]:
# # Define the similarity metric
# def similarity(house1, house2):
#     # Calculate the cosine similarity between two houses
#     return cosine_similarity(
#         house1.values.reshape(1, -1), house2.values.reshape(1, -1)
#     )[0][0]


# # Define the recommendation function
# def recommend(query_house, n=10):
#     # Initialize an empty list to store the similarity scores
#     scores = []

#     # Loop through all the houses in the dataset
#     for i in range(len(df)):
#         # Get the current house
#         house = df.iloc[i]

#         # Calculate the similarity score with the query house
#         score = similarity(query_house, house)

#         # Append the score and the index to the list
#         scores.append((score, i))

#     # Sort the list by the score in descending order
#     scores = sorted(scores, reverse=True)

#     # Get the top n results
#     results = scores[:n]
#     dfs = []

#     # Print the results
#     print(f"Top {n} recommendations for the query house:")
#     for score, i in results:
#         # Get the house from the dataset
#         house = data[columns].iloc[i]

#         # Print the house features and the score
#         # print(f"House {i}:")
#         house_df = house.to_frame().T
#         house_df["Community"] = data["Community"]
#         house_df["ML_Number"] = data["ML_Number"]
#         # print(house.to_frame().T)
#         print(f"Similarity Score: {score:.4f}")
#         print()
#         dfs.append(house_df)

#     # Concatenate the DataFrames in the list
#     result_df = pd.concat(dfs)

#     return result_df


# query_house = df.iloc[0]
# nn_df = recommend(query_house, n=8)

# # all_recommendations = pd.DataFrame()

# # for i in range(len(df)):
# #     query_house = df.iloc[i]
# #     nn_df = recommend(query_house, n=8)
# #     all_recommendations = pd.concat([all_recommendations, nn_df])

# # # # Reset index of the resulting DataFrame
# # # all_recommendations.reset_index(drop=True, inplace=True)

# # # Display or further process the all_recommendations DataFrame as needed
# # all_recommendations.to_csv("NN.csv")
# nn_df.to_csv("NN.csv")
# print(query_house)
# nn_df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import datetime

# Assuming df is your dataset containing house information, and 'sold_date' is the column representing the date when the house was sold.
# You might need to adjust the column names based on your actual dataset structure.


# Define the similarity metric
def similarity(house1, house2, date_range=30):
    # Calculate the cosine similarity between two houses
    cosine_sim = cosine_similarity(
        house1.drop("sold_date").values.reshape(1, -1),
        house2.drop("sold_date").values.reshape(1, -1),
    )[0][0]

    # Check if sold dates fall within the specified date range
    date_diff = abs((house1["Sold_Month"] - house2["Sold_Month"]).days)
    date_similarity = 1 / (1 + date_diff / date_range)

    # Combine cosine similarity and date similarity
    combined_similarity = (cosine_sim + date_similarity) / 2

    return combined_similarity


# Define the recommendation function
def recommend(query_house, n=10, date_range=30):
    # Initialize an empty list to store the similarity scores
    scores = []

    # Loop through all the houses in the dataset
    for i in range(len(df)):
        # Get the current house
        house = df.iloc[i]

        # Calculate the similarity score with the query house
        score = similarity(query_house, house, date_range)

        # Append the score and the index to the list
        scores.append((score, i))

    # Sort the list by the score in descending order
    scores = sorted(scores, reverse=True)

    # Get the top n results
    results = scores[:n]
    dfs = []

    # Print the results
    print(f"Top {n} recommendations for the query house:")
    for score, i in results:
        # Get the house from the dataset
        house = data[columns].iloc[i]

        # Print the house features and the score
        house_df = house.to_frame().T
        house_df["Community"] = data["Community"]
        house_df["ML_Number"] = data["ML_Number"]
        print(f"Similarity Score: {score:.4f}")
        print()
        dfs.append(house_df)

    # Concatenate the DataFrames in the list
    result_df = pd.concat(dfs)

    return result_df


# Example usage:
query_house = df.iloc[0]
nn_df = recommend(query_house, n=8, date_range=30)

# Save the result to a CSV file
nn_df.to_csv("NN.csv")

print(query_house)
print(nn_df)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming df is your original dataset and data is the dataset containing house features
# Replace 'columns' with the actual column names you want to consider in your similarity calculation


# Define the similarity metric for content-based filtering
def content_similarity(house1, house2):
    return cosine_similarity(
        house1.values.reshape(1, -1), house2.values.reshape(1, -1)
    )[0][0]


# Define the content-based recommendation function
def content_based_recommend(query_house, n=10):
    scores = []

    for i in range(len(df)):
        house = df.iloc[i]
        score = content_similarity(query_house, house)
        scores.append((score, i))

    scores = sorted(scores, reverse=True)
    results = scores[:n]

    return results


# Define the collaborative recommendation function (example: item-based collaborative filtering)
def collaborative_recommend(query_house, n=10):
    # Implement collaborative filtering logic based on user-item interactions
    # This could involve identifying houses that users with similar preferences have interacted with
    # For simplicity, let's assume collaborative_recommend returns a list of house indices

    # Replace this with your collaborative filtering logic
    collaborative_results = [
        (0.8, 0),
        (0.7, 1),
        (0.6, 2),
        (0.5, 3),
        (0.4, 4),
        (0.3, 5),
        (0.2, 6),
        (0.1, 7),
        (0.05, 8),
        (0.01, 9),
    ]

    return collaborative_results


# Define the hybrid recommendation function
def hybrid_recommend(query_house, n=10, content_weight=0.7, collaborative_weight=0.3):
    # Get content-based recommendations
    content_results = content_based_recommend(query_house, n=n)

    # Get collaborative recommendations
    collaborative_results = collaborative_recommend(query_house, n=n)

    # Combine the recommendations with weights
    hybrid_results = []
    for (content_score, content_index), (collab_score, collab_index) in zip(
        content_results, collaborative_results
    ):
        hybrid_score = (
            content_weight * content_score + collaborative_weight * collab_score
        )
        hybrid_results.append((hybrid_score, content_index))

    # Sort the hybrid results by score in descending order
    hybrid_results = sorted(hybrid_results, reverse=True)
    results = hybrid_results[:n]

    dfs = []

    # Print the results
    print(f"Top {n} recommendations for the query house:")
    for score, i in results:
        # Get the house from the dataset
        house = data[columns].iloc[i]

        # Print the house features and the score
        # print(f"House {i}:")
        house_df = house.to_frame().T
        house_df["Community"] = data["Community"]
        house_df["ML_Number"] = data["ML_Number"]
        # print(house.to_frame().T)
        print(f"Similarity Score: {score:.4f}")
        print()
        dfs.append(house_df)

    # Concatenate the DataFrames in the list
    result_df = pd.concat(dfs)

    return result_df


# Example usage:
query_house = df.iloc[0]  # Replace with your actual query house
hybrid_results = hybrid_recommend(query_house, n=10)

# Print or further process the hybrid_results as needed
hybrid_results.to_csv("NN1.csv")